In [68]:
from tree2 import Tree
from anytree import Node, RenderTree
import sys
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
import site
from copy import deepcopy
import re
import string
import zss

suffixList=["d","ll","re","ve","s","t"]

def setPosTag(postag,form):
    #print(postag)
    if(postag=="^+G"):
        return "Z"
    if(postag=="^+O"):
        return "Z"
    if(postag=="N+G"):
        return "S"
    if(postag=="^+V"):
        return "Z"
    if(postag=="^+R"):
        return "V"
    if(postag=="N+V"):
        return "L"
    if(postag=="O+V"):
        return "L"
    if(postag=="V+R"):
        return "V"
    if(postag=="R+V"):
        return "L"
    if(postag=="V+V"):
        return "V"
    if(postag=="X+R"):
        return "Y"
    if ((form in suffixList)& (form !="s")):
        return "L"
    elif ((form in suffixList)& (form =="s")):
        if(postag=="V+O"):
            return "L"
        else:
            return "Z"
    

def lookup(postag):
    rplc=""
    if(postag=="NN" or postag=="NNS"):
        rplc="N"
    elif(postag=="PRP" or postag=="WP"):
        rplc="O"
    elif(postag=="NNP" or postag=="NNPS"):
        rplc="^"
    elif(postag[0]=="V" or postag=="MD"):
        rplc="V"
    elif(postag[0]=="J"):
        rplc="A"
    elif(postag[0]=="R" or postag=="WRB"):
        rplc="R"
    elif(postag=="UH" or postag=="UH"):
        rplc="!"
    elif(postag=="IN" or postag=="TO"):
        rplc="P"
    elif(postag=="CC" or postag=="CC"):
        rplc="&"
    elif(postag=="RP"):
        rplc="T"
    elif(postag=="EX" or postag=="PDT"):
        rplc="X"
    elif(postag=="CD"):
        rplc="$"
    elif(postag=="FW" or postag=="POS" or postag=="SYM" or postag=="LS"):
        rplc="G"
    elif(postag in string.punctuation or postag=="''" or postag=="``" or postag=="-LRB-" or postag=="-RRB-"):
        rplc=","
    elif(postag=="HT"):
        rplc="#"
    else:
        answer=re.match(r'WP$',postag)
        answer2=re.match(r'PRP$',postag)
        if(answer!=None or answer2!=None or postag=="DT" or postag=="WDT" or postag=="PRP$"):
            rplc="D"
    if(rplc==""):
        rplc=postag
    return rplc

def set_parentID(mergee,merger):
    wordIDlist=[mergee[0],merger[0]]
    parentID=-999
    #print(str(mergee[0])+" "+str(merger[0]))
    #print(str(mergee[6])+" "+str(merger[6]))
    if (mergee[6]==merger[6]):
        parentID= merger[6]
    else:
        if ((mergee[6] not in wordIDlist)&(merger[6] in wordIDlist)):
            parentID= mergee[6]
        elif ((merger[6] not in wordIDlist)&(mergee[6] in wordIDlist)):
            parentID= merger[6]
        else:
            parentID= min(merger[6],mergee[6])
    #print("The parentID is "+str(parentID))
    return parentID
            
line_holders=[]
tree_holders=[]
pos_holder={}
holder_pos_holder=[]
with open("tweet_sample_outputs.txt",encoding="utf-8") as f1:
    hold=f1.read()
    trees=list(filter(None,hold.split("\n*****************************************************\n")))
    for tree in trees:
        tree_holders.append(tree.split("\n\n"))
print  (len(tree_holders))      

i=0

tweet_tree_holder=[]   
for trees in tree_holders:
    tree_b_holder=[]    #tweet level container
    for tree in trees:
        b_holder=[]   #tree level container
        i=0
        merge_position=[]
        sentences=tree.split("\n")
        j=0
        while j in range(len(sentences)):
            if sentences[j]!="":
                b=sentences[j].split("\t")
                b[3]= lookup(b[3])
                b[4]= b[3]
                b_holder.append(b) 
                if(b[1].endswith("'")):
                        merge_b=sentences[j+1].split("\t")
                        merge_b[3]= lookup(merge_b[3])
                        merge_b[4]= merge_b[3]
                        #print(b[1])
                        if not((b[3]=="^")&(merge_b[3]=="^")):
                            if(merge_b[1] in suffixList):    
                            #print("||"+merge_b[1])
                                new_form=b_holder[i][1]+merge_b[1]
                                new_upostag=b_holder[i][3]+'+'+merge_b[3]
                                b_holder[i][1]=new_form
                                #print(new_form+" ")
                                b_holder[i][3]=setPosTag(new_upostag,merge_b[1])
                                #print(str(b_holder[i][3])+'\n---')
                                b_holder[i][6]=set_parentID(b_holder[i],merge_b)
                                merge_position.append(merge_b[0])
                                j=j+1
                i=i+1
            j=j+1
        #print(str(len(b_holder))+"===>"+str(merge_position))
        if((len(b_holder)>0) & (len(merge_position)>0)):
            for b in b_holder:
                if(b[6] in merge_position):
                    b[6]=str(int(b[6])-1)
                    #print (True)
        tree_b_holder.append(b_holder)
    tweet_tree_holder.append(tree_b_holder)

for tree_b_holder in tweet_tree_holder:
    for b_holder in tree_b_holder:
        for line in b_holder:
            print(line)
file_to_write=open("regular_processed_input.txt","w",encoding="utf-8")
str1=""

print(len(tweet_tree_holder))
for tree_b_holder in tweet_tree_holder:
    for b_holder in tree_b_holder:
        #print(b_holder)
        for line in b_holder:
            str1='\t'.join(line)
            file_to_write.write(str1)
            file_to_write.write("\n")
    file_to_write.write("\n*****************************************************\n")
        
file_to_write.close()

100
['1', 'NEW', '_', '^', '^', '_', '2', 'NMOD']
['2', 'REPORT', '_', '^', '^', '_', '3', 'SUB']
['3', 'remains', '_', 'V', 'V', '_', '0', 'ROOT']
['4', 'one', '_', '$', '$', '_', '3', 'PRD']
['5', 'of', '_', 'P', 'P', '_', '4', 'NMOD']
['6', 'the', '_', 'D', 'D', '_', '7', 'NMOD']
['7', "continent's", '_', 'L', 'N', '_', '11', 'SUB']
['9', 'most', '_', 'R', 'R', '_', '10', 'AMOD']
['10', 'buoyant', '_', 'A', 'A', '_', '11', 'NMOD']
['11', 'economies', '_', 'N', 'N', '_', '5', 'PMOD']
['12', 'with', '_', 'P', 'P', '_', '11', 'NMOD']
['13', 'a', '_', 'D', 'D', '_', '15', 'NMOD']
['14', 'growth', '_', 'N', 'N', '_', '15', 'NMOD']
['15', 'rate', '_', 'N', 'N', '_', '12', 'PMOD']
['16', 'of', '_', 'P', 'P', '_', '15', 'NMOD']
['17', '79', '_', '$', '$', '_', '16', 'PMOD']
['18', 'in', '_', 'P', 'P', '_', '15', 'NMOD']
['19', '2016', '_', '$', '$', '_', '18', 'PMOD']
['1', 'Investing', '_', 'V', 'V', '_', '0', 'ROOT']
['2', 'BitCoins', '_', 'N', 'N', '_', '1', 'OBJ']
['3', 'via', '_', 'P',